<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/Language_BME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備  


必要なパッケージをインストールする．

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 24 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

あらかじめダウンロードしていたテキストファイルを，Google Colabの「ファイル」にドラッグする．  
この講義では次のファイルを用意している．


*   Neko.txt: 夏目漱石「吾輩は猫である」
*   yokaidan.txt: 井上円了「妖怪談」
  
※いずれも[青空文庫](https://www.aozora.gr.jp)より，著作権消滅済み．


# N-gram  

"Ngram"は，キーワードの出現頻度を可視化する意味で用いられることが多い．  
[Google Ngram Viewer](https://books.google.com/ngrams) (Google Books)  
[NDL Ngram Viewer](https://lab.ndl.go.jp/ngramviewer/) (国立国会図書館)  

## N-gramをもとにして文章を生成してみる  
参考：[3. Pythonによる自然言語処理　1-1. 単語N-gram](https://qiita.com/y_itoh/items/82222af50bf1f80255eb) @y_itoh(yumi ito)  
  
この方法は純粋に訓練データ(テキスト)の単語の統計情報から文章を生成するものである．

**--------別のテキストファイルで試すときはここから実行して！--------**

Google Colabにダウンロード済みのテキストファイル(**Neko.txt**)を読み込む．  
他のテキストにしたい場合は，たとえばyokaidan.txtにする場合は，次のようにファイル名を変更して実行する．  


```
with open('yokaidan.txt')
```


読み込んだテキストが表示される．

In [ ]:
with open('Neko.txt', mode='rt', encoding='utf-8') as f:
    read_text = f.read()
nekotxt = read_text

print(nekotxt)

### 分かち書き

MeCabで分かち書きし，結果を表示する．

In [ ]:
import MeCab
tagger = MeCab.Tagger("-Owakati")
nekotxt = tagger.parse(nekotxt)

# print(nekotxt)
nekotxt = nekotxt.split()
print(nekotxt)

### 辞書の作成

N-gram辞書を生成する．  
連続する2語を要素とし，その頻度をカウントしたものがdic2(2-gram)，3語をカウントしたものがdic3(3-gram)である．

In [ ]:
from collections import Counter
import numpy as np
from numpy.random import *

string = nekotxt

# 除外する文字記号
delimiter = ['「', '」', '…', '　']

# 2語のリスト
double = list(zip(string[:-1], string[1:]))
double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)

# 3語のリスト
triple = list(zip(string[:-2], string[1:-1], string[2:]))
triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)

# 要素数をカウントして辞書を生成
dic2 = Counter(double)
dic3 = Counter(triple)

2-gram辞書を表示する．  
長いので中断してしまうが問題ない．  
  
例えば，

```
('聞き', 'たまえ') 3
```
は，「聞き」の直後に「たまえ」があるパターンが3回見つかった，という意味である．


In [ ]:
for u,v in dic2.items():
    print(u, v)

3-gram辞書を表示する．  
これも長いので中断してしまうが問題ない．  
  
同じく，例えば，

```
('飛び', '上っ', 'て') 2
```
は，「飛び」「上っ」「て」の連続パターンが2回見つかった，という意味である．


In [ ]:
for u,v in dic3.items():
    print(u, v)

### 文章生成

文章生成の関数を定義する．

In [ ]:
def nextword(words, dic):
    ## ➀先頭の単語wordsの要素数gramsを取得
    grams = len(words)

    ## ➁N-gram辞書dicから一致する要素を抽出
    # 2語の場合
    if grams == 2:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]), #1番目が合致
            dic.items())))
    # 3語の場合
    else:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]) and (lambda x: x[0][1] == words[2]), #1番目と2番目が合致
            dic.items())))

    ## ➂一致する語がない場合のエラーメッセージ
    if(len(matcheditems) == 0):
        print("No matched generator for", words[1])
        return ''

    ## ➃重み付き出現頻度リスト
    # matcheditemsから出現頻度を取得
    probs = [row[1] for row in matcheditems]
    # 0～1の疑似乱数を生成して出現頻度にかける
    weightlist = rand(len(matcheditems)) * probs

    ## ➄matcheditemsから重み付き出現頻度が最大の要素を取得
    if grams == 2:
        u = matcheditems[np.argmax(weightlist)][0][1]
    else:
        u = matcheditems[np.argmax(weightlist)][0][2]
    return u

文章生成を試す．  

1)まずはそのまま実行すると，「吾輩」からはじまる文章が生成される．これは2-gram辞書を用いている．実行するたびに結果が変わるので，何度か試すとよい．  
2)次に2行目の冒頭に半角の#を書き加え，逆に3行目の冒頭の#を消去して実行する．今度は3-gram辞書を用いている．これも実行のたびに結果が変わる．  

```
#words = ['', '吾輩'] # 2-gram
words = ['', '吾輩', 'は'] # 3-gram
```
3)最後に好きな単語で文章生成する．たとえば，「吾輩は」の代わりに「猫が」にしてみる．
```
words = ['', '猫', 'が'] # 3-gram
```



  

In [ ]:
# 先頭の単語wordsを入力
words = ['', '吾輩'] # 2-gram
#words = ['', '吾輩', 'は'] # 3-gram

# 出力outputの先頭にwordsを埋め込む
output = words[1:]

# ｢次の語｣を取得
for i in range(100):
    # 2語の場合
    if len(words) == 2:
        newword = nextword(words, dic2)
    # 3語の場合
    else:
        newword = nextword(words, dic3)

    # 出力outputに次の語を追加
    output.append(newword)
    # 次の文字が終止符なら終了
    if newword in ['', '。', '？', '！']:
        break
    # 次のnextwordの準備
    words = output[-len(words):]
    print(words)

# 出力outputを表示
for u in output:
    print(u, end='')


# 文書単語行列と文章類似度  
参考：  
* [scikit-learnのCountVectorizerやTfidfVectorizerの日本語での使い方について](https://qiita.com/kiyuka/items/3de09e313a75248ca029)　@kiyuka
* [Pythonで文書類似度算出！](https://toukei-lab.com/python-mecab)　ウマたん  

## 文書単語行列

次の4つの文章の文書単語行列を求める．あらかじめ分かち書きする．  
* これは最初のドキュメントです。
* このドキュメントは2番目のドキュメントです。
* そして、これは3番目のものです。
* これは最初のドキュメントですか?
  
単に出現頻度が表になっているだけなので難しくはない．

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display
corpus = [
    ['これ', 'は', '最初', 'の', 'ドキュメント', 'です', '。'],
    ['この', 'ドキュメント', 'は', '2', '番目', 'の', 'ドキュメント', 'です', '。'],
    ['そして', '、', 'これ', 'は', '3', '番目', 'の', 'もの', 'です', '。'],
    ['これ', 'は', '最初', 'の', 'ドキュメント', 'です', 'か', '?']
]
vectorizer = CountVectorizer(analyzer=lambda x: x)

vec = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()
df = pd.DataFrame(vec.toarray(), columns=feature_names)
display(df)

,2,3,?,、,。,か,この,これ,そして,です,の,は,もの,ドキュメント,最初,番目
0,0,0,0,0,1,0,0,1,0,1,1,1,0,1,1,0
1,1,0,0,0,1,0,1,0,0,1,1,1,0,2,0,1
2,0,1,0,1,1,0,0,1,1,1,1,1,1,0,0,1
3,0,0,1,0,0,1,0,1,0,1,1,1,0,1,1,0


## 二つのURLのテキストの類似度を求める  

### とりあえずやってみる

In [9]:
import requests
from bs4 import BeautifulSoup
import sys
import MeCab
from time import sleep
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

##関数定義
# Step1：URLからテキスト情報をスクレイピング
def geturl(urls):
    all_text=[]
    for url in urls:
        r=requests.get(url)
        c=r.content
        soup=BeautifulSoup(c,"html.parser")
        article1_content=soup.find_all("p")
        temp=[]
        for con in article1_content:
            out=con.text
            temp.append(out)
        text=''.join(temp)
        all_text.append(text)
        sleep(1)
    return all_text

# Step2：それらをMeCabで形態素解析。名詞だけ抽出。
def mplg(article):
    word_list = ""
    m=MeCab.Tagger()
    m1=m.parse (text)
    for row in m1.split("\n"):
        word =row.split("\t")[0]#タブ区切りになっている１つ目を取り出す。ここには形態素が格納されている
        if word == "EOS":
            break
        else:
            pos = row.split("\t")[1]#タブ区切りになっている2つ目を取り出す。ここには品詞が格納されている
            slice = pos[:2]
            if slice == "名詞":
                word_list = word_list +" "+ word
    return word_list

# Step3：名詞の出現頻度からTF-IDF/COS類似度を算出。テキスト情報のマッチ度を測る
def tfidf(word_list):
    docs = np.array(word_list)#Numpyの配列に変換する
    #単語を配列ベクトル化して、TF-IDFを計算する
    vecs = TfidfVectorizer(
                token_pattern=u'(?u)\\b\\w+\\b'#文字列長が 1 の単語を処理対象に含めることを意味します。
                ).fit_transform(docs)
    vecs = vecs.toarray()
    return vecs


def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

##実装
word_list=[]
texts=geturl(["https://toukei-lab.com/conjoint","https://toukei-lab.com/correspondence"])
for text in texts:
    word_list.append(mplg(text))

vecs=tfidf(word_list)
print(cossim(vecs[1],vecs[0]))

0.4005091643186147


最後に数字が1つ表示されたが，これが次の二つのURL(ソースコードでは59行目付近に記載)のコサイン類似度である．  
**全く同じ文章の場合，1.0**となる．  
* https://toukei-lab.com/conjoint
* https://toukei-lab.com/correspondence

### 名詞の抽出

中身を解説する．MeCabで分かち書きを行うところまでは同じだが，今回は名詞だけを抽出する．結果を見てみる．

In [13]:
print('最初のURLの分かち書きの結果(名詞のみ): 単語数{}'.format(len(word_list[0])))
print(word_list[0])
print('二番目のURLの分かち書きの結果(名詞のみ): 単語数{}'.format(len(word_list[1])))
print(word_list[1])

最初のURLの分かち書きの結果(名詞のみ): 単語数1171
 データ サイエンティスト ウマ たん (@ statistics 1012 ) 消費 者 商品 機能 の 場合 コンジョイント 分析 マーケティング 非常 重要 役割 記事 コンジョイント 分析 目次 商品 サービス 部分 改善 消費 者 の 効用 値 大小 把握 ため 手法 コンジョイント 分析 どこ スペック の 価格 どこ 許容 の 消費 者 明確 意識 わけ 潜在 的 感覚 消費 者 潜在 的 効用 把握 ため 直接的 機能 の 様々 スペック 商品 点数 こと 機能 効用 値 算出 の コンジョイント 分析 コンジョイント 分析 こと 消費 者 最適 スペック 把握 こと それ ムダ 機能 拡張 こと 消費 者 機能 価格 商品 サービス 提供 こと の コンジョイント 分析 気 の 商品 プロファイル 作成 方法 商品 プロファイル 適当 作成 要因 交絡 結果 実験 計画 法 直交 表 割り当て 商品 プロファイル 作成 コンジョイント 分析 R 実装 パッケージ conjoint tea データセット プロファイル データ コンジョイント 分析 tprof 以下 よう プロファイル データ tlevn それぞれ 水準 データ の 商品 プロファイル 3 水準 3 水準 3 水準 2 水準 price 水準 low / medium / high variety 水準 black / green / red kind 水準 bags / granulated / leafty aroma 水準 yes / no 最後 それら 効用 値 100 人 分 データ tprefm これら データ コンジョイント 分析 自動的 グラフ プロット これ 要因 ごと 寄与 率 variety 寄与 率 若干 価格 部分 効用 値 low 効用 値 の 感覚 一致 high medium 比較 時 medium 方 効用 値 の コンジョイント 分析 最後 簡単 コンジョイント 分析 商品 サービス 機能 特徴 受容 度 分析 手法 マーケティング 商品 開発 R 簡単 実装 可能 コンジョイント 分析 方 以下 書籍 変量 解析 コンジョイント 分析 以外 様々 手法 変量 解析 手法 以下 記事 マー

### 文書単語行列の表示

文書単語行列を表示する．  
各々の文章の文書ベクトル(tf-idf)の中身を見ると，二つの文章で異なることがわかる．  
もし同じ文章であれば二つのベクトルは全く同じであり，なす角度は0度なので，コサイン類似度cos(0)=1.0となる．

In [17]:
print('ベクトルのサイズ: {}'.format(len(vecs[0])))
print('\n最初のURLの名詞のtf-idf')
print(vecs[0])
print('\n二番目のURLのの名詞のtf-idf')
print(vecs[1])

ベクトルのサイズ: 228

最初のURLの名詞のtf-idf
[0.         0.03213922 0.02286732 0.         0.03213922 0.06860197
 0.         0.02286732 0.         0.         0.03213922 0.03213922
 0.03213922 0.         0.03213922 0.         0.         0.03213922
 0.03213922 0.06427845 0.         0.         0.         0.03213922
 0.02286732 0.03213922 0.06427845 0.         0.09641767 0.03213922
 0.02286732 0.03213922 0.02286732 0.04573465 0.03213922 0.
 0.         0.02286732 0.03213922 0.03213922 0.03213922 0.03213922
 0.02286732 0.         0.06427845 0.03213922 0.02286732 0.02286732
 0.02286732 0.         0.02286732 0.13720394 0.02286732 0.02286732
 0.03213922 0.         0.03213922 0.03213922 0.03213922 0.06427845
 0.02286732 0.06427845 0.25154055 0.02286732 0.03213922 0.
 0.         0.         0.         0.02286732 0.03213922 0.
 0.41780992 0.04573465 0.         0.02286732 0.02286732 0.06860197
 0.02286732 0.02286732 0.02286732 0.02286732 0.09641767 0.
 0.02286732 0.18293858 0.02286732 0.02286732 0.02286732 0.0228